In [25]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import wordnet
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split,GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [26]:
pl_19 = pd.read_csv('result1_prohet_forecasted1_min.csv')
fl_19 = pd.read_excel('result_fuel_forecasted.xlsx')

In [27]:
train = pd.read_excel('Data_Train.xlsx')
target=train.Price
del train['Price']
print(train.shape)


test = pd.read_excel('Test_set.xlsx')
print(test.shape)


original_df=pd.concat([train,test])
original_df.reset_index(drop=True,inplace=True)
print(original_df.shape)

(10683, 10)
(2671, 10)
(13354, 10)


In [28]:
original_df['Date_of_Journey'] = pd.to_datetime(original_df['Date_of_Journey'], format='%d/%m/%Y')
original_df['Day_week_of_Journey'] =original_df['Date_of_Journey'].apply(lambda time: time.dayofweek)
original_df['Day_of_Journey'] = original_df['Date_of_Journey'].apply(lambda time: time.day)
original_df['Month_of_Journey'] = original_df['Date_of_Journey'].apply(lambda time: time.month)

original_df['Dep_Time'] = pd.to_datetime(original_df['Dep_Time'])
original_df['Dep_Time_Hour']=original_df['Dep_Time'].apply(lambda time: time.hour)
original_df['Dep_Time_Hour']=pd.to_numeric(original_df['Dep_Time_Hour'])



original_df.loc[(original_df['Dep_Time_Hour']<6) , 'MMT_fg_dept'] = '0'
original_df.loc[(original_df['Dep_Time_Hour']>=6) & (original_df['Dep_Time_Hour']<12), 'MMT_fg_dept'] = '1'
original_df.loc[(original_df['Dep_Time_Hour']>=12) & (original_df['Dep_Time_Hour']<18), 'MMT_fg_dept'] = '2'
original_df.loc[(original_df['Dep_Time_Hour']>=18) , 'MMT_fg_dept'] = '3'


original_df.loc[(original_df['Dep_Time_Hour']<11) , 'goibibo_fg_dept'] = '0'
original_df.loc[(original_df['Dep_Time_Hour']>=11) & (original_df['Dep_Time_Hour']<17), 'goibibo_fg_dept'] = '1'
original_df.loc[(original_df['Dep_Time_Hour']>=17) & (original_df['Dep_Time_Hour']<21), 'goibibo_fg_dept'] = '2'
original_df.loc[(original_df['Dep_Time_Hour']>=21) , 'goibibo_fg_dept'] = '3'



original_df['Arrival_Time'] = original_df['Arrival_Time'].str.strip()
original_df['Arrival_Time_v1'] = original_df['Arrival_Time'].astype(str).str[:6]
original_df['Arrival_Time_v1'] = original_df['Arrival_Time_v1'].str.strip()

original_df['Arrival_Time_v1'] = pd.to_datetime(original_df['Arrival_Time_v1'])
original_df['Arr_Time_Hour']=original_df['Arrival_Time_v1'].apply(lambda time: time.hour)
original_df['Arr_Time_Hour']=pd.to_numeric(original_df['Arr_Time_Hour'])


original_df.loc[(original_df['Arr_Time_Hour']<6) , 'MMT_fg_arr'] = '0'
original_df.loc[(original_df['Arr_Time_Hour']>=6) & (original_df['Arr_Time_Hour']<12), 'MMT_fg_arr'] = '1'
original_df.loc[(original_df['Arr_Time_Hour']>=12) & (original_df['Arr_Time_Hour']<18), 'MMT_fg_arr'] = '2'
original_df.loc[(original_df['Arr_Time_Hour']>=18) , 'MMT_fg_arr'] = '3'


original_df.loc[(original_df['Arr_Time_Hour']<11) , 'goibibo_fg_arr'] = '0'
original_df.loc[(original_df['Arr_Time_Hour']>=11) & (original_df['Arr_Time_Hour']<17), 'goibibo_fg_arr'] = '1'
original_df.loc[(original_df['Arr_Time_Hour']>=17) & (original_df['Arr_Time_Hour']<21), 'goibibo_fg_arr'] = '2'
original_df.loc[(original_df['Arr_Time_Hour']>=21) , 'goibibo_fg_arr'] = '3'



original_df['Total_Stops']=original_df['Total_Stops'].str.replace('[^0-9]','')
original_df['Total_Stops']=original_df['Total_Stops'].str.replace('[^\w\s]','')
original_df['new_column_qua']=original_df['Total_Stops'].str.len()
original_df['Total_Stops']=np.where(original_df['new_column_qua']==0, 0,original_df['Total_Stops'])


del original_df['new_column_qua']
del original_df['Arrival_Time_v1']
del original_df['Dep_Time']
del original_df['Date_of_Journey']



original_df['Arrival_Time_v2']=original_df['Arrival_Time'].str.replace('[^A-Za-z]','')
original_df['Arrival_Time_v2']=original_df['Arrival_Time_v2'].str.replace('[^\w\s]','')
original_df['new_column_qua1']=original_df['Arrival_Time_v2'].str.len()
original_df['Days_fg']=np.where(original_df['new_column_qua1']==0, 0,1)


del original_df['new_column_qua1']
del original_df['Arrival_Time_v2']
del original_df['Arrival_Time']



original_df['Duration1']=original_df['Duration'].str.replace(' ', '')
original_df['Hour']=original_df['Duration1'].str.split('[^0-9]').apply(lambda x: int(x[0]) * 60)
original_df.loc[(original_df['Duration1'].astype(str).str[-3:].str.contains('m')==True), 'Minute']=original_df['Duration1'].astype(str).str[-3:]
original_df['Minute']=original_df['Minute'].str.replace('[^0-9]','')
original_df['Minute']=pd.to_numeric(original_df.Minute)
original_df.Minute = original_df.Minute.fillna(0)
original_df['Tot_Duration']=original_df['Hour'] + original_df['Minute']

del original_df['Hour']
del original_df['Minute']
del original_df['Duration1']
del original_df['Duration']


original_df['Route1']=original_df['Route'].str.replace('[^A-Za-z]',' ')
original_df['Route2']=original_df['Route1'].astype(str).str[:3].replace('[A-Za-z]',' ')
original_df['Route3']=original_df['Route'].astype(str).str[-3:].replace('[A-Za-z]',' ')

original_df['Route3'] = original_df['Route3'].str.strip()
original_df['Route2'] = original_df['Route2'].str.strip()
original_df['Route1'] = original_df['Route1'].str.strip()


for i in range(13354):
    if (pd.isnull(original_df['Route1'][i])==False):
        original_df.at[i,'Route4']=re.sub(original_df['Route2'][i],'', original_df['Route1'][i])
        original_df.at[i,'Route5']=re.sub(original_df['Route3'][i],'', original_df['Route4'][i])

original_df['Route5'] = original_df['Route5'].str.strip()        

del original_df['Route']
del original_df['Route1']
del original_df['Route2']
del original_df['Route3']
del original_df['Route4']

original_df['Route5']=original_df['Route5'].str.replace('[^\w\s]','')
original_df.Route5 = original_df.Route5.fillna('NaN')


vec = CountVectorizer()
X = vec.fit_transform(original_df.Route5)
msc_df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
msc_df=pd.DataFrame(msc_df)

original_df1=pd.concat([original_df,msc_df], axis=1)
del original_df1['Route5']
del original_df1['nan']

original_df1.Total_Stops = original_df1.Total_Stops.fillna(0)

#original_df1['Day_week_of_Journey'] = original_df1['Day_week_of_Journey'].apply(str)
#original_df1['Day_of_Journey'] = original_df1['Day_of_Journey'].apply(str)
#original_df1['Month_of_Journey'] = original_df1['Month_of_Journey'].apply(str)
#original_df1['Dep_Time_Hour'] = original_df1['Dep_Time_Hour'].apply(str)
#original_df1['Arr_Time_Hour'] = original_df1['Arr_Time_Hour'].apply(str)
#original_df1['Days_fg'] = original_df1['Days_fg'].apply(str)

original_df1['Destination']=original_df1['Destination'].str.replace('New Delhi', 'Delhi')
#print(original_df1.head())
#print(original_df1.Destination.unique())
#print(original_df1.Source.unique())

original_df2=original_df1.copy(deep=True)

#pl_19['Month_of_Journey'] = pl_19['Month_of_Journey'].apply(str)
original_df2 = pd.merge(original_df2, pl_19,  how='left', on=['Airline','Month_of_Journey'])

#fl_19['Month_of_Journey'] = fl_19['Month_of_Journey'].apply(str)
original_df2 = pd.merge(original_df2, fl_19,  how='left', on=['Month_of_Journey'])



In [29]:
original_df2['weekend'] = np.where(original_df2['Day_week_of_Journey']>=5, 1, 0)

In [30]:
original_df2['Total_Stops']=original_df2['Total_Stops'].astype(np.float)
original_df2.P_load = original_df2.P_load.fillna(-999)

In [31]:
categorical = ["Airline", "Source", "Destination", "Additional_Info", "MMT_fg_dept", "goibibo_fg_dept", "MMT_fg_arr", "goibibo_fg_arr"]

label_encoder = LabelEncoder()
for col in categorical:
    original_df2[col] = label_encoder.fit_transform(original_df2[col].astype(str))

train_mod1=original_df2.iloc[:train.shape[0]]
test_mod1=original_df2.iloc[train.shape[0]:]

X_train, X_validation, y_train, y_validation = train_test_split(train_mod1, target, train_size=0.7, random_state=1234)

In [76]:
model=xgb.XGBRegressor(colsample_bylevel=0.9, 
                       colsample_bytree=0.9,
                       gamma=2, 
                       learning_rate=0.01, #0.01
                       max_depth=60, #60
                       min_child_weight=2, #2
                       n_estimators=440, #440
                       objective='reg:linear', 
                       reg_alpha=1, 
                       reg_lambda=1,
                       scale_pos_weight=1, 
                       seed=99, 
                       subsample=0.9,
                       silent=True)
#model.fit(X_train, y_train)
model.fit(train_mod1, target)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.9, gamma=2, learning_rate=0.01, max_delta_step=0,
       max_depth=60, min_child_weight=2, missing=None, n_estimators=440,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=1, reg_lambda=1, scale_pos_weight=1, seed=99, silent=True,
       subsample=0.9)

In [77]:
ypred = model.predict(X_train)
ypred_val = model.predict(X_validation)

print(sqrt(mean_squared_error(y_train, ypred)))
print(sqrt(mean_squared_error(y_validation, ypred_val)))

542.0741231296357
540.6995630668697


In [78]:
y_pred_test2=model.predict(test_mod1)
y_pred_test2=pd.DataFrame(y_pred_test2)
y_pred_test2.columns=['Price']
y_pred_test2.to_excel('submission_XGBoost2.xlsx', index=False)#0.9500006

In [79]:
#437.6478614135423
#434.30277026851786

In [80]:
# save model to file
pickle.dump(model, open("xgb_final.pickle.dat", "wb"))

<h> **stacking** </h>

In [81]:
y_pred_test2=model.predict(test_mod1)
y_pred_test2=pd.DataFrame(y_pred_test2)
y_pred_test2.columns=['xgb']
y_pred_test2.to_excel('submission_XGBoost_stck_val.xlsx', index=False)

In [82]:
y_pred_test2=model.predict(train_mod1)
y_pred_test2=pd.DataFrame(y_pred_test2)
y_pred_test2.columns=['xgb']
y_pred_test2.to_excel('submission_XGBoost_stck_train.xlsx', index=False)